In [2]:
import pickle
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
import re


def cleaning(sentence):
    stop_words = set(stopwords.words('english'))
    tokens = re.sub(r'[^\w\s]', '', sentence.lower()).replace("\n", " ").split(" ")
    cleaned = [token for token in tokens if token not in stop_words]
    return " ".join(cleaned)


labels = ["insurance-etc","investment", "medical-sales", "phising", "sexual", "software-sales"]
text = []
classes = []
# for label in labels:
#     path = "data/raw/spam/Annotated/"+label
  
#     os.chdir(path)
  
#     def read_text_file(file_path):
#         with open(file_path, 'r', encoding='ISO-8859-1') as f:
#             return f.read()
      
#     for file in os.listdir():
    
#         if file.endswith(".txt"):
#             file_path = f"{path}/{file}"
#             text.append(cleaning(str(read_text_file(file_path))))
#             classes.append(label)

for cla in labels:
    all_files = os.listdir("data/raw/spam/Annotated/" + cla)
    for fil in all_files:
        if fil.endswith(".txt"):
            file_path = "data/raw/spam/Annotated/" + cla + "/" + fil
            with open(file_path, 'r', encoding='ISO-8859-1') as f:
                text.append(cleaning(str(f.read())))
                classes.append(cla)
                        
data = pd.DataFrame({'sentence':text, 'label':classes})
data

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xiangzairan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/xiangzairan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/xiangzairan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


sentence           label
0     subject take advantage low interest  rates  no...   insurance-etc
1     subject lowest rate us history sound  let 6000...   insurance-etc
2     subject application pre  approved monday oct 1...   insurance-etc
3     subject final offer wed  29 sep 2004 19  16  0...   insurance-etc
4     subject  congratulations conditional apprgm ov...   insurance-etc
...                                                 ...             ...
2337  subject popular software low low prices  bury ...  software-sales
2338  subject clean ur computer 3 ey 85  chance comp...  software-sales
2339  subject software incredibly low prices  73  lo...  software-sales
2340  subject latest qem software available  low pri...  software-sales
2341  subject cnet reviews  results favorite program...  software-sales

[2342 rows x 2 columns]

In [3]:
def tfidf(word):
    sentence = data['sentence']
    idf = np.log(len(sentence)/sentence.str.contains(word).sum())
    result = []
    for i in range(len(sentence)):
        tf = sentence.iloc[i].count(word)/(len(sentence.iloc[i]))
        result.append(tf*idf)
    return result


In [4]:
import json

f = open("data/out/seedwords.json")
seeds = json.load(f)

# f = open('/Users/lorraine/Desktop/Spam_Filter_old/seedwords.json')
# seeds = json.load(f)
result = pd.DataFrame()
for key, value in seeds.items():
    df = pd.DataFrame()
    for w in value:
        df[w] = tfidf(w)
    result[key] = df.sum(axis = 1)
result


insurance-etc  investment  medical-sales   phising    sexual  \
0          0.032417    0.012053       0.000000  0.000000  0.007130   
1          0.057250    0.003856       0.000000  0.000000  0.000000   
2          0.015633    0.000000       0.001337  0.000000  0.000000   
3          0.012803    0.006197       0.000000  0.003688  0.000000   
4          0.002356    0.001836       0.000359  0.000000  0.000995   
...             ...         ...            ...       ...       ...   
2337       0.000000    0.003640       0.000000  0.000000  0.000000   
2338       0.000000    0.000000       0.003057  0.000000  0.000000   
2339       0.000000    0.003788       0.002119  0.000000  0.000000   
2340       0.000000    0.002512       0.000000  0.010819  0.000000   
2341       0.000000    0.003755       0.000000  0.002510  0.000000   

      software-sales  
0           0.000000  
1           0.000000  
2           0.000000  
3           0.022694  
4           0.000902  
...              ...  
2337        0.008796  
2338        0.000000  
2339        0.009155  
2340        0.078401  
2341        0.014344  

[2342 rows x 6 columns]

In [5]:
data["prediction"] = result.idxmax(1)
data

sentence           label  \
0     subject take advantage low interest  rates  no...   insurance-etc   
1     subject lowest rate us history sound  let 6000...   insurance-etc   
2     subject application pre  approved monday oct 1...   insurance-etc   
3     subject final offer wed  29 sep 2004 19  16  0...   insurance-etc   
4     subject  congratulations conditional apprgm ov...   insurance-etc   
...                                                 ...             ...   
2337  subject popular software low low prices  bury ...  software-sales   
2338  subject clean ur computer 3 ey 85  chance comp...  software-sales   
2339  subject software incredibly low prices  73  lo...  software-sales   
2340  subject latest qem software available  low pri...  software-sales   
2341  subject cnet reviews  results favorite program...  software-sales   

          prediction  
0      insurance-etc  
1      insurance-etc  
2      insurance-etc  
3     software-sales  
4      insurance-etc  
...              ...  
2337  software-sales  
2338   medical-sales  
2339  software-sales  
2340  software-sales  
2341  software-sales  

[2342 rows x 3 columns]

In [6]:
# micro and macro F1 using tf-idf
from sklearn import metrics

In [7]:
metrics.f1_score(data["label"], data["prediction"], average="micro")

0.6874466268146883

In [8]:
metrics.f1_score(data["label"], data["prediction"], average="macro")

0.6618372754126397

In [10]:
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

def preprocessing(sentence):
    tokens = sentence.split(" ")
    return [token for token in tokens if token!="" and token != " "]
features = data["sentence"].apply(preprocessing)
model = Word2Vec(sentences=features, vector_size=110, window=5, min_count=1, workers=8)
#features

In [11]:
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")
model.train(features, total_examples=len(data), epochs=800)
#vector = model.wv["atheism"]
#vector

(270776739, 274806400)

In [12]:
def get_vectors_per_label(filename):
    f = open(filename)
    seeds = json.load(f)
    vector_per_label = []
    for key, value in seeds.items():
        lst = []
        for w in value:
            lst.append(model.wv[w])
        arr = np.asarray(lst)
        total = np.average(arr, axis=0)
        vector_per_label.append(total)
    return vector_per_label

vector_per_label = get_vectors_per_label('data/out/seedwords.json')

In [13]:
def get_vector_per_doc(feature):
    vector_per_doc = []
    for feat in feature:
        lst = []
        for w in feat:
            lst.append(model.wv[w])
        arr = np.asarray(lst)
        total = np.average(arr, axis=0)
        vector_per_doc.append(total)
    return vector_per_doc
vector_per_doc = get_vector_per_doc(features)

In [14]:
len(vector_per_doc)

2342

In [15]:
f = open('test/seedwords.json')
seeds = json.load(f)
from numpy.linalg import norm
def predict_word2vec(vector_per_doc, vector_per_label):
    predictions = []
    labels = list(seeds.keys())
    for doc in vector_per_doc:
        cosine = []
        for label in vector_per_label:
            cosine.append(np.dot(doc,label)/(norm(doc)*norm(label)))
        max_value = max(cosine)
        max_index = cosine.index(max_value)
        predictions.append(labels[max_index])
    return predictions   
prediction_word2vec = predict_word2vec(vector_per_doc, vector_per_label)

In [16]:
data["prediction_word2vec"] = prediction_word2vec
data

sentence           label  \
0     subject take advantage low interest  rates  no...   insurance-etc   
1     subject lowest rate us history sound  let 6000...   insurance-etc   
2     subject application pre  approved monday oct 1...   insurance-etc   
3     subject final offer wed  29 sep 2004 19  16  0...   insurance-etc   
4     subject  congratulations conditional apprgm ov...   insurance-etc   
...                                                 ...             ...   
2337  subject popular software low low prices  bury ...  software-sales   
2338  subject clean ur computer 3 ey 85  chance comp...  software-sales   
2339  subject software incredibly low prices  73  lo...  software-sales   
2340  subject latest qem software available  low pri...  software-sales   
2341  subject cnet reviews  results favorite program...  software-sales   

          prediction prediction_word2vec  
0      insurance-etc       insurance-etc  
1      insurance-etc       insurance-etc  
2      insurance-etc       insurance-etc  
3     software-sales       insurance-etc  
4      insurance-etc      software-sales  
...              ...                 ...  
2337  software-sales       medical-sales  
2338   medical-sales      software-sales  
2339  software-sales       medical-sales  
2340  software-sales      software-sales  
2341  software-sales      software-sales  

[2342 rows x 4 columns]

In [17]:
# micro and macro F1 using word2vec
metrics.f1_score(data["label"], data["prediction_word2vec"], average="micro")

0.7045260461144321

In [18]:
metrics.f1_score(data["label"], data["prediction_word2vec"], average="macro")

0.7187069493140096

In [19]:
sum(data["prediction_word2vec"] == data["label"])/len(data)

0.7045260461144321

In [ ]:
import fasttext
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.4)
with open('spam-train.txt', 'w', encoding="utf-8") as f:
    for idx, row in train_set.iterrows():
        f.write("__label__" + row.label + " " + row.sentence + "\n")
model = fasttext.train_unsupervised(input='spam-train.txt', epoch=600, lr=0.05, wordNgrams=4, loss='hs', dim=40)
def get_vectors_per_label_fasttext(filename):
    f = open(filename)
    seeds = json.load(f)
    vector_per_label = []
    for key, value in seeds.items():
        lst = []
        for w in value:
            lst.append(model.get_word_vector(w))
        arr = np.asarray(lst)
        total = np.average(arr, axis=0)
        vector_per_label.append(total)
    return vector_per_label
vector_per_label = get_vectors_per_label_fasttext('test/seedwords.json')
def get_vector_per_doc_fasttext(feature):
    vector_per_doc = []
    for feat in feature:
        lst = []
        for w in feat:
            lst.append(model.get_word_vector(w))
        arr = np.asarray(lst)
        total = np.average(arr, axis=0)
        vector_per_doc.append(total)
    return vector_per_doc
vector_per_doc = get_vector_per_doc_fasttext(features)
def predict_fasttext(vector_per_doc, vector_per_label):
    predictions = []
    labels = list(seeds.keys())
    for doc in vector_per_doc:
        cosine = []
        for label in vector_per_label:
            cosine.append(np.dot(doc,label)/(norm(doc)*norm(label)))
        max_value = max(cosine)
        max_index = cosine.index(max_value)
        predictions.append(labels[max_index])
    return predictions   
prediction_fasttext = predict_fasttext(vector_per_doc, vector_per_label)
data["prediction_fasttext"] = prediction_fasttext

Read 0M words
Number of words:  11296
Number of labels: 9
Progress:  53.8% words/sec/thread:  101025 lr:  0.023100 avg.loss:  2.772606 ETA:   0h 3m41s5.7% words/sec/thread:  115967 lr:  0.047141 avg.loss:  6.393805 ETA:   0h 6m33s  8.3% words/sec/thread:  114374 lr:  0.045845 avg.loss:  6.525591 ETA:   0h 6m27s 29.5% words/sec/thread:  102861 lr:  0.035237 avg.loss:  4.737868 ETA:   0h 5m31s 103240 lr:  0.031819 avg.loss:  4.075275 ETA:   0h 4m58s 43.5% words/sec/thread:  102914 lr:  0.028235 avg.loss:  3.420052 ETA:   0h 4m25s

In [52]:
print(model.labels)

['subject', 'company', 'statements', '_', 'com', 'information', 'may', 'please', '2', 'email', 'within', '1', 'http', 'e', 'forward', 'us', 'money', 'price', 'time', '3', 'securities', '0', 'get', '</s>', '000', 'new', '5', 'news', 'stock', 'account', 'one', 'looking', 'business', 'investment', 'report', 'number', 'inc', 'home', 'act', 'next', 'security', 'international', 'future', 'part', 'r', '00', 'software', 'million', 'need', 'based', '2004', 'mail', 'advice', 'made', 'oo', '6', 'c', 'l', 'bank', 'also', 'lottery', 'could', 'use', '10', '4', '7', 'contact', 'professional', 'online', 'adobe', 'name', 'performance', 'b', 'best', 'go', 'section', 'address', 'due', 'events', 'products', 'results', 'mr', 'www', 'companies', 'funds', 'free', 'order', 'stocks', 'shares', 'expectations', 'want', 'fact', 'program', 'make', 'day', 'claim', '8', 'save', 'p', 'service', 'current', 'net', 'market', 'stop', 'th', 'would', 'numbers', 'today', 'interest', 'world', 'office', 'like', 'newsletter', 

In [ ]:
metrics.f1_score(data["label"], data["prediction_fasttext"], average="micro")

In [ ]:
metrics.f1_score(data["label"], data["prediction_fasttext"], average="macro")